In [1]:
###packages
import fiona
import numpy as np
import pandas as pd 
import os
from pyproj import Proj, transform
import folium
import geopandas as gpd
from shapely.geometry import Point
from tqdm import tqdm

In [4]:
###acc_data : 사고 데이터, code_info: 사고 데이터 컬럼 정보
acc_data = pd.read_csv("/home/yh_zoo/data/ACC_INFO.csv", sep="\t")
code_info = pd.read_excel("/home/yh_zoo/data/사고_코드정보.xls")

display(acc_data.head(), code_info.head())

/tmp/ipykernel_2954/122504631.py:2: DtypeWarning: Columns (8,9,65,67) have mixed types. Specify dtype option on import or set low_memory=False.
  acc_data = pd.read_csv("/home/yh_zoo/data/ACC_INFO.csv", sep="\t")


,ACC_YM,ORG_CD,ACC_NO,ACC_YMD,ACC_TME,DAY_CD,BJD_CD,HIT_BJD_CD,ACC_PLC,ACC_STU,...,ESTB_CLSF_CD,LMT_SPD,BUS_CNT_CAR_RD_YN,ONE_TRF_YN,LOT_NO,DWG_228,ACC_TYP_MAP_CD,ACC_FTR_CD,ETL_RGST_ID,ETL_REG_DT
0,201702,0,2017021000100477,20170210,2000,6,2650010300,A,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6,7,ETLDSADM,2015-01-01 00:00:00.000000
1,201702,0,2017020800100500,20170208,2200,4,4311410200,A,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6,4,ETLDSADM,2015-01-01 00:00:00.000000
2,201702,0,2017020800100487,20170208,2100,4,3017010600,A,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,7,7,ETLDSADM,2015-01-01 00:00:00.000000
3,201702,0,2017020800100161,20170208,1000,4,4577034032,A,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3,5,ETLDSADM,2015-01-01 00:00:00.000000
4,201702,0,2017020900100382,20170209,1800,5,4719012000,A,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4,1,ETLDSADM,2015-01-01 00:00:00.000000


,CD_ID,CD_NM,CD_ID.1,CD_VAL,CD_CPN,HGH_CD_VAL,HGH_CD_CPN,HGH_CD_CLS_VAL,HGH_CD_CLS_CPN,HGH_CD_LARCLS_VAL,HGH_CD_LARCLS_CPN,READ_ORD
0,ACC_FTR_CD,사고_요인_코드,ACC_FTR_CD,1,보행자,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ACC_FTR_CD,사고_요인_코드,ACC_FTR_CD,2,동물,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ACC_FTR_CD,사고_요인_코드,ACC_FTR_CD,3,"나무,지주등",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ACC_FTR_CD,사고_요인_코드,ACC_FTR_CD,4,자전거이용자,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ACC_FTR_CD,사고_요인_코드,ACC_FTR_CD,5,이륜자동차,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
####column 
print("column 갯수 : ",len(acc_data.columns))
acc_data.columns

column 갯수 :  90


Index(['ACC_YM', 'ORG_CD', 'ACC_NO', 'ACC_YMD', 'ACC_TME', 'DAY_CD', 'BJD_CD',
       'HIT_BJD_CD', 'ACC_PLC', 'ACC_STU', 'ACC_TYP_CD', 'LAW_VIO_CD',
       'TM_X_CRD', 'TM_Y_CRD', 'GRS_X_CRD', 'GRS_Y_CRD', 'POL_RPT_YN',
       'WEA_STA_CD', 'RD_CLS_CD', 'RD_TYP_CD', 'BIG_ACC_YN', 'ACC_GRD_CD',
       'DEATH_CNT', 'SERI_CNT', 'SLTWD_CNT', 'WND_CNT', 'DOBJ_CNT', 'DMG_AMT',
       'OLD_CHLD_INC_YN', 'DRK_ACC_YN', 'RUN_ACC_YN', 'UNLIC_ACC_YN',
       'HIT_ACC_CRD', 'LINK_X_CRD', 'LINK_Y_CRD', 'IMG_DIR_NM', 'IMG_FILE_NM',
       'TWV_ACC_YN', 'HAM_TWV_ACC_YN', 'VCT_TWV_ACC_YN', 'MOTR_ACC_YN',
       'HAM_MOTR_ACC_YN', 'VCT_MOTR_ACC_YN', 'BCL_ACC_YN', 'HAM_BCL_ACC_YN',
       'VCT_BCL_ACC_YN', 'CT_EQUIP_ACC_YN', 'HAM_CT_EQUIP_ACC_YN',
       'VCT_CT_EQUIP_ACC_YN', 'AC_EQUIP_ACC_YN', 'HAM_AC_EQUIP_ACC_YN',
       'VCT_AC_EQUIP_ACC_YN', 'WKR_ACC_YN', 'HAM_WKR_ACC_YN', 'VCT_WKR_ACC_YN',
       'CHLD_ACC_YN', 'HAM_CHLD_ACC_YN', 'VCT_CHLD_ACC_YN', 'OLD_ACC_YN',
       'HAM_OLD_ACC_YN', 'VCT_OLD_

In [6]:
code_info["CD_ID_NM"] = code_info["CD_ID"] + code_info["CD_NM"]

set(code_info["CD_ID_NM"])

{'ACC_FTR_CD사고_요인_코드',
 'ACC_GRD_CD사고_등급_코드',
 'ACC_TYP_CD사고_유형_코드',
 'ACC_TYP_MAP_CD사고_형태_지도_코드',
 'AGE_CD연령_코드',
 'BUS_CNT_CAR_RD_YN버스_중앙_차로_여부',
 'CAR_CLSF_CD차량_종별_코드',
 'DRV_LIC_CLSF_CD운전_면허_종별_코드',
 'ESTB_CLSF_CD시설물_종별_코드',
 'HIT_BJD_CDHIT_법정동_코드',
 'LAW_VIO_CD법규_위반_코드',
 'LINK_CLSF_CD링크_종별_코드',
 'ONE_TRF_YN일반_통행_여부',
 'ORG_CD수집기관_코드',
 'RD_CLS_CD도로_종류_코드',
 'RD_GRD_CD도로_등급_코드',
 'RD_TYP_CD도로_형태_코드',
 'SEX_CD성별_코드',
 'WEA_STA_CD기상_상태_코드',
 'YN여부'}

### 전체 columns 설명

- ACC_YM : 년월 
- ORG_CD : 수집기관 코드(보험사,조합)
- ACC_NO : 사고고유번호
- ACC_YMD : 년월일 
- ACC_TIME : 시간 
- DAY_CD : 요일 
- BJD_CD : 법정동 코드
- ACC_PLC : 사고장소
- ACC_STU : ?
- ACC_TYP_CD : 사고유형코드 
- LAW_VIO_CD : 법규 위반 코드 
- TM_X_CRD : x좌표
- TM_Y_CRD : y좌표
- GRS_X_CRD : x좌표
- GRS_Y_CRD : y좌표
- POL_RPT_YN : ?
- WEA_STA_CD : 기상_상태_코드
- RD_CLS_CD :  도로 종류 코드 
- RD_TYP_CD : 도로 형태 코드 
- BIG_ACC_YN : ?
- ACC_GRD_CD : 사고 등급 코드 
- DEATH_CNT : 사망자 수 
- SERI_CNT : ?
- SLTWD_CNT : ?
- WND_CNT : ?
- DOBJ_CNT : ?
- DMG_AMT : ?
- OLD_CHLD_INC_YN : ?
- DRK_ACC_YN : ?
- RUN_ACC_YN : ?
- UNLIC_ACC_YN : ?
- HIT_ACC_CRD : ?
- LINK_X_CRD : ?
- LINK_Y_CRD : ?
- IMG_DIR_NM : ?
- IMG_FILE_NM : ?
- TWV_ACC_YN : ?
- HAM_TWV_ACC_YN : ?
- VCT_TWV_ACC_YN : ?
- MOTR_ACC_YN : ?
- HAM_MOTR_ACC_YN : ? 
- VCT_MOTR_ACC_YN : ?
- BCL_ACC_YN : ?
- HAM_BCL_ACC_YN : ?
- VCT_BCL_ACC_YN : ?
- CT_EQUIP_ACC_YN : ?
- HAM_CT_EQUIP_ACC_YN : ?
- VCT_CT_EQUIP_ACC_YN : ?
- AC_EQUIP_ACC_YN : ?
- HAM_AC_EQUIP_ACC_YN : ?
- VCT_AC_EQUIP_ACC_YN : ?
- WKR_ACC_YN : ?
- HAM_WKR_ACC_YN : ? 
- VCT_WKR_ACC_YN : ?
- CHLD_ACC_YN : ?
- HAM_CHLD_ACC_YN : ? 
- VCT_CHLD_ACC_YN : ?
- OLD_ACC_YN : ?
- HAM_OLD_ACC_YN : ?
- VCT_OLD_ACC_YN : ?
- MEDLAR_ACC_YN : ?
- MEDLAR_PSGT_YN : ?
- MEDLAR_TRK_YN : ?
- CAR_CLSF_CD : ?
- CAR_USE_CD : ?
- DRV_LIC_CRR : ?
- DRV_LIC_CLSF_CD :운전_면허_종별_코드
- SEX_CD : 성별_코드
- AGE_CD : 연령_코드
- FORN_YN_CD : ?
- NODE_ID : ?
- LINK_ID : ?
- RD_NO : ?
- RD_NM : ?
- RD_PAV_YN : ? 
- RD_MED_YN : ?
- TOT_LANE_CNT : ?
- RD_GRD_CD : 도로_등급_코드
- LINK_CLSF_CD : 링크_종별_코드
- ESTB_CLSF_CD : 시설물_종별_코드
- LMT_SPD : ?
- BUS_CNT_CAR_RD_YN : 버스_중앙_차로_여부
- ONE_TRF_YN : 일반_통행_여부
- LOT_NO : ?
- DWG_228 : ?
- ACC_TYP_MAP_CD : 사고_형태_지도_코드
- ACC_FTR_CD : 사고_요인_코드
- ETL_RGST_ID : ? 
- ETL_REG_DT : ?

### 사용 columns 설명 
- ACC_YMD : 년월일
- ACC_TME : 시간 
- DAY_CD : 요일
- BJD_CD : 법정동 코드 (3000000000 대전)
- ACC_TYP_CD : 사고유형
- LAW_VIO_CD : 법규위반
- TM_X_CRD : x
- TM_Y_CRD : y
- WEA_STA_CD : 기상 상태 코드 
- RD_CLS_CD : 도로 종류 코드
- RD_TYP_CD : 도로 형태 코드 
- ACC_GRD_CD : 사고 등급 코드 
- DEATH_CNT : 사망자수 
- SEX_CD : 성별 
- AGE_CD : 나이

In [7]:
###truck data filter 
truck_acc_data = acc_data.loc[acc_data["CAR_CLSF_CD"] == 3]

###columns filter
truck_acc_data = truck_acc_data[['ACC_YMD', 'ACC_TME', 'DAY_CD', 'BJD_CD','ACC_TYP_CD','LAW_VIO_CD',
                                 'TM_X_CRD', 'TM_Y_CRD','WEA_STA_CD', 'RD_CLS_CD', 'RD_TYP_CD', 
                                 'ACC_GRD_CD','DEATH_CNT','SEX_CD', 'AGE_CD']]

###region filter
DJ_truck_acc_data = truck_acc_data.loc[(truck_acc_data["BJD_CD"] >= 3000000000) & (truck_acc_data["BJD_CD"] < 3023012700)].reset_index(drop=True)

display(DJ_truck_acc_data.shape, DJ_truck_acc_data.head())

(3155, 15)

,ACC_YMD,ACC_TME,DAY_CD,BJD_CD,ACC_TYP_CD,LAW_VIO_CD,TM_X_CRD,TM_Y_CRD,WEA_STA_CD,RD_CLS_CD,RD_TYP_CD,ACC_GRD_CD,DEATH_CNT,SEX_CD,AGE_CD
0,20170211,1300,7,3017012800,220,7.0,989233.0,1819155.0,1,3.0,8.0,3,0,01,56
1,20160228,300,1,3014011700,310,6.0,989940.0,1812496.0,1,3.0,2.0,3,0,01,23
2,20160205,300,6,3020012400,210,2.0,989191.0,1819695.0,1,3.0,1.0,3,0,01,56
3,20160202,2200,3,3020010200,210,2.0,983593.0,1812737.0,1,3.0,8.0,3,0,01,32
4,20160204,800,5,3020013900,240,6.0,983408.0,1821596.0,1,3.0,8.0,3,0,01,33


In [ ]:
###좌표 변환 epsg5179 -> epsg 4326
proj_UTMK = Proj(init='epsg:5179')
proj_WGS84 = Proj(init='epsg:4326')

def transform_utmk_to_w84(df):
    return pd.Series(transform(proj_UTMK, proj_WGS84, df['TM_X_CRD'], df['TM_Y_CRD']), index=['x', 'y'])

DJ_truck_acc_data[['TM_Y_CRD','TM_X_CRD']] = DJ_truck_acc_data.apply(transform_utmk_to_w84, axis=1)

DJ_truck_acc_data.to_csv("data/prepared_acc_data.csv",index=False)

- DBSCAN

In [25]:
#https://www.webucator.com/article/python-color-constants-module/
colors = {-1: 'red', 0: '#FAEBD7', 1: '#FFEFDB', 2: '#EEDFCC', 3: '#CDC0B0', 4: '#8B8378', 5: '#00FFFF', 6: '#7FFFD4', 7: '#76EEC6', 8: '#66CDAA', 9: '#458B74', 10: '#F0FFFF', 
          11: '#E0EEEE', 12: '#C1CDCD', 13: '#838B8B', 14: '#E3CF57', 15: '#F5F5DC', 16: '#FFE4C4', 17: '#EED5B7', 18: '#CDB79E', 19: '#8B7D6B', 20: '#000000', 
          21: '#FFEBCD', 22: '#0000FF', 23: '#0000EE', 24: '#0000CD', 25: '#00008B', 26: '#8A2BE2', 27: '#9C661F', 28: '#A52A2A', 29: '#FF4040', 30: '#EE3B3B', 
          31: '#CD3333', 32: '#8B2323', 33: '#DEB887', 34: '#FFD39B', 35: '#EEC591', 36: '#CDAA7D', 37: '#8B7355', 38: '#8A360F', 39: '#8A3324', 40: '#5F9EA0', 
          41: '#98F5FF', 42: '#8EE5EE', 43: '#7AC5CD', 44: '#53868B', 45: '#FF6103', 46: '#FF9912', 47: '#ED9121', 48: '#7FFF00', 49: '#76EE00', 50: '#66CD00',
          51: '#458B00', 52: '#D2691E', 53: '#FF7F24', 54: '#EE7621', 55: '#CD661D', 56: '#8B4513', 57: '#3D59AB', 58: '#3D9140', 59: '#808A87', 60: '#FF7F50', 
          61: '#FF7256', 62: '#EE6A50', 63: '#CD5B45', 64: '#8B3E2F', 65: '#6495ED', 66: '#FFF8DC', 67: '#EEE8CD', 68: '#CDC8B1', 69: '#8B8878', 70: '#DC143C', 
          71: '#00EEEE', 72: '#00CDCD', 73: '#008B8B', 74: '#B8860B', 75: '#FFB90F', 76: '#EEAD0E', 77: '#CD950C', 78: '#8B6508', 79: '#A9A9A9', 80: '#006400', 
          81: '#BDB76B', 82: '#556B2F', 83: '#CAFF70', 84: '#BCEE68', 85: '#A2CD5A', 86: '#6E8B3D', 87: '#FF8C00', 88: '#FF7F00', 89: '#EE7600', 90: '#CD6600', 
          91: '#8B4500', 92: '#9932CC', 93: '#BF3EFF', 94: '#B23AEE', 95: '#9A32CD', 96: '#68228B', 97: '#E9967A', 98: '#8FBC8F', 99: '#C1FFC1', 100: '#B4EEB4', 
          101: '#9BCD9B', 102: '#698B69', 103: '#483D8B', 104: '#2F4F4F', 105: '#97FFFF', 106: '#8DEEEE', 107: '#79CDCD', 108: '#528B8B', 109: '#00CED1', 110: '#9400D3', 
          111: '#FF1493', 112: '#EE1289', 113: '#CD1076', 114: '#8B0A50', 115: '#00BFFF', 116: '#00B2EE', 117: '#009ACD', 118: '#00688B', 119: '#696969', 120: '#1E90FF',
          121: '#1C86EE', 122: '#1874CD', 123: '#104E8B', 124: '#FCE6C9', 125: '#00C957', 126: '#B22222', 127: '#FF3030', 128: '#EE2C2C', 129: '#CD2626', 130: '#8B1A1A',
          131: '#FF7D40', 132: '#FFFAF0', 133: '#228B22', 134: '#DCDCDC', 135: '#F8F8FF', 136: '#FFD700', 137: '#EEC900', 138: '#CDAD00', 139: '#8B7500', 140: '#DAA520',
          141: '#FFC125', 142: '#EEB422', 143: '#CD9B1D', 144: '#8B6914', 145: '#808080', 146: '#030303', 147: '#1A1A1A', 148: '#1C1C1C', 149: '#1F1F1F', 150: '#212121',
          151: '#242424', 152: '#262626', 153: '#292929', 154: '#2B2B2B', 155: '#2E2E2E', 156: '#303030', 157: '#050505', 158: '#333333', 159: '#363636', 160: '#383838',
          161: '#3B3B3B', 162: '#3D3D3D', 163: '#404040', 164: '#424242', 165: '#454545', 166: '#474747', 167: '#4A4A4A', 168: '#080808', 169: '#4D4D4D', 170: '#4F4F4F',
          171: '#525252', 172: '#545454', 173: '#575757', 174: '#595959', 175: '#5C5C5C', 176: '#5E5E5E', 177: '#616161', 178: '#636363', 179: '#0A0A0A', 180: '#666666',
          181: '#6B6B6B', 182: '#6E6E6E', 183: '#707070', 184: '#737373', 185: '#757575', 186: '#787878', 187: '#7A7A7A', 188: '#7D7D7D', 189: '#0D0D0D', 190: '#7F7F7F',
          191: '#828282', 192: '#858585', 193: '#878787', 194: '#8A8A8A', 195: '#8C8C8C', 196: '#8F8F8F', 197: '#919191', 198: '#949494', 199: '#969696', 200: '#0F0F0F',
          201: '#999999', 202: '#9C9C9C', 203: '#9E9E9E', 204: '#A1A1A1', 205: '#A3A3A3', 206: '#A6A6A6', 207: '#A8A8A8', 208: '#ABABAB', 209: '#ADADAD', 210: '#B0B0B0',
          211: '#121212', 212: '#B3B3B3', 213: '#B5B5B5', 214: '#B8B8B8', 215: '#BABABA', 216: '#BDBDBD', 217: '#BFBFBF', 218: '#C2C2C2', 219: '#C4C4C4', 220: '#C7C7C7',
          221: '#C9C9C9', 222: '#141414', 223: '#CCCCCC', 224: '#CFCFCF', 225: '#D1D1D1', 226: '#D4D4D4', 227: '#D6D6D6', 228: '#D9D9D9', 229: '#DBDBDB', 230: '#DEDEDE',
          231: '#E0E0E0', 232: '#E3E3E3', 233: '#171717', 234: '#E5E5E5', 235: '#E8E8E8', 236: '#EBEBEB', 237: '#EDEDED', 238: '#F0F0F0', 239: '#F2F2F2', 240: '#F7F7F7',
          241: '#FAFAFA', 242: '#FCFCFC', 243: '#008000', 244: '#00FF00', 245: '#00EE00', 246: '#00CD00', 247: '#008B00', 248: '#ADFF2F', 249: '#F0FFF0', 250: '#E0EEE0',
          251: '#C1CDC1', 252: '#838B83', 253: '#FF69B4', 254: '#FF6EB4', 255: '#EE6AA7', 256: '#CD6090', 257: '#8B3A62', 258: '#CD5C5C', 259: '#FF6A6A', 260: '#EE6363',
          261: '#CD5555', 262: '#8B3A3A', 263: '#4B0082', 264: '#FFFFF0', 265: '#EEEEE0', 266: '#CDCDC1', 267: '#8B8B83', 268: '#292421', 269: '#F0E68C', 270: '#FFF68F',
          271: '#EEE685', 272: '#CDC673', 273: '#8B864E', 274: '#E6E6FA', 275: '#FFF0F5', 276: '#EEE0E5', 277: '#CDC1C5', 278: '#8B8386', 279: '#7CFC00', 280: '#FFFACD',
          281: '#EEE9BF', 282: '#CDC9A5', 283: '#8B8970', 284: '#ADD8E6', 285: '#BFEFFF', 286: '#B2DFEE', 287: '#9AC0CD', 288: '#68838B', 289: '#F08080', 290: '#E0FFFF',
          291: '#D1EEEE', 292: '#B4CDCD', 293: '#7A8B8B', 294: '#FFEC8B', 295: '#EEDC82', 296: '#CDBE70', 297: '#8B814C', 298: '#FAFAD2', 299: '#D3D3D3', 300: '#FFB6C1',
          301: '#FFAEB9', 302: '#EEA2AD', 303: '#CD8C95', 304: '#8B5F65', 305: '#FFA07A', 306: '#EE9572', 307: '#CD8162', 308: '#8B5742', 309: '#20B2AA', 310: '#87CEFA',
          311: '#B0E2FF', 312: '#A4D3EE', 313: '#8DB6CD', 314: '#607B8B', 315: '#8470FF', 316: '#778899', 317: '#B0C4DE', 318: '#CAE1FF', 319: '#BCD2EE', 320: '#A2B5CD',
          321: '#6E7B8B', 322: '#FFFFE0', 323: '#EEEED1', 324: '#CDCDB4', 325: '#8B8B7A', 326: '#32CD32', 327: '#FAF0E6', 328: '#FF00FF', 329: '#EE00EE', 330: '#CD00CD',
          331: '#8B008B', 332: '#03A89E', 333: '#800000', 334: '#FF34B3', 335: '#EE30A7', 336: '#CD2990', 337: '#8B1C62', 338: '#BA55D3', 339: '#E066FF', 340: '#D15FEE',
          341: '#B452CD', 342: '#7A378B', 343: '#9370DB', 344: '#AB82FF', 345: '#9F79EE', 346: '#8968CD', 347: '#5D478B', 348: '#3CB371', 349: '#7B68EE', 350: '#00FA9A',
          351: '#48D1CC', 352: '#C71585', 353: '#E3A869', 354: '#191970', 355: '#BDFCC9', 356: '#F5FFFA', 357: '#FFE4E1', 358: '#EED5D2', 359: '#CDB7B5', 360: '#8B7D7B',
          361: '#FFE4B5', 362: '#FFDEAD', 363: '#EECFA1', 364: '#CDB38B', 365: '#8B795E', 366: '#000080', 367: '#FDF5E6', 368: '#808000', 369: '#6B8E23', 370: '#C0FF3E',
          371: '#B3EE3A', 372: '#9ACD32', 373: '#698B22', 374: '#FF8000', 375: '#FFA500', 376: '#EE9A00', 377: '#CD8500', 378: '#8B5A00', 379: '#FF4500', 380: '#EE4000',
          381: '#CD3700', 382: '#8B2500', 383: '#DA70D6', 384: '#FF83FA', 385: '#EE7AE9', 386: '#CD69C9', 387: '#8B4789', 388: '#EEE8AA', 389: '#98FB98', 390: '#9AFF9A',
          391: '#90EE90', 392: '#7CCD7C', 393: '#548B54', 394: '#BBFFFF', 395: '#AEEEEE', 396: '#96CDCD', 397: '#668B8B', 398: '#DB7093', 399: '#FF82AB', 400: '#EE799F',
          401: '#CD6889', 402: '#8B475D', 403: '#FFEFD5', 404: '#FFDAB9', 405: '#EECBAD', 406: '#CDAF95', 407: '#8B7765', 408: '#33A1C9', 409: '#FFC0CB', 410: '#FFB5C5',
          411: '#EEA9B8', 412: '#CD919E', 413: '#8B636C', 414: '#DDA0DD', 415: '#FFBBFF', 416: '#EEAEEE', 417: '#CD96CD', 418: '#8B668B', 419: '#B0E0E6', 420: '#800080',
          421: '#9B30FF', 422: '#912CEE', 423: '#7D26CD', 424: '#551A8B', 425: '#872657', 426: '#C76114', 427: '#FF0000', 428: '#EE0000', 429: '#CD0000', 430: '#8B0000',
          431: '#BC8F8F', 432: '#FFC1C1', 433: '#EEB4B4', 434: '#CD9B9B', 435: '#8B6969', 436: '#4169E1', 437: '#4876FF', 438: '#436EEE', 439: '#3A5FCD', 440: '#27408B',
          441: '#FA8072', 442: '#FF8C69', 443: '#EE8262', 444: '#CD7054', 445: '#8B4C39', 446: '#F4A460', 447: '#308014', 448: '#54FF9F', 449: '#4EEE94', 450: '#43CD80',
          451: '#2E8B57', 452: '#FFF5EE', 453: '#EEE5DE', 454: '#CDC5BF', 455: '#8B8682', 456: '#5E2612', 457: '#8E388E', 458: '#C5C1AA', 459: '#71C671', 460: '#555555',
          461: '#1E1E1E', 462: '#282828', 463: '#515151', 464: '#5B5B5B', 465: '#848484', 466: '#8E8E8E', 467: '#B7B7B7', 468: '#C1C1C1', 469: '#EAEAEA', 470: '#F4F4F4',
          471: '#7D9EC0', 472: '#AAAAAA', 473: '#8E8E38', 474: '#C67171', 475: '#7171C6', 476: '#388E8E', 477: '#A0522D', 478: '#FF8247', 479: '#EE7942', 480: '#CD6839',
          481: '#8B4726', 482: '#C0C0C0', 483: '#87CEEB', 484: '#87CEFF', 485: '#7EC0EE', 486: '#6CA6CD', 487: '#4A708B', 488: '#6A5ACD', 489: '#836FFF', 490: '#7A67EE',
          491: '#6959CD', 492: '#473C8B', 493: '#708090', 494: '#C6E2FF', 495: '#B9D3EE', 496: '#9FB6CD', 497: '#6C7B8B', 498: '#FFFAFA', 499: '#EEE9E9', 500: '#CDC9C9',
          501: '#8B8989', 502: '#00FF7F', 503: '#00EE76', 504: '#00CD66', 505: '#008B45', 506: '#4682B4', 507: '#63B8FF', 508: '#5CACEE', 509: '#4F94CD', 510: '#36648B',
          511: '#D2B48C', 512: '#FFA54F', 513: '#EE9A49', 514: '#CD853F', 515: '#8B5A2B', 516: '#008080', 517: '#D8BFD8', 518: '#FFE1FF', 519: '#EED2EE', 520: '#CDB5CD',
          521: '#8B7B8B', 522: '#FF6347', 523: '#EE5C42', 524: '#CD4F39', 525: '#8B3626', 526: '#40E0D0', 527: '#00F5FF', 528: '#00E5EE', 529: '#00C5CD', 530: '#00868B',
          531: '#00C78C', 532: '#EE82EE', 533: '#D02090', 534: '#FF3E96', 535: '#EE3A8C', 536: '#CD3278', 537: '#8B2252', 538: '#808069', 539: '#F5DEB3', 540: '#FFE7BA',
          541: '#EED8AE', 542: '#CDBA96', 543: '#8B7E66', 544: '#FFFFFF', 545: '#F5F5F5', 546: '#FFFF00', 547: '#EEEE00', 548: '#CDCD00', 549: '#8B8B00'}

In [26]:
DJ_truck_acc_data = pd.read_csv("/home/yh_zoo/data/prepared_acc_data.csv")

In [27]:
#https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html
from sklearn.cluster import DBSCAN

x = DJ_truck_acc_data["TM_X_CRD"].values.reshape(-1,1)
y = DJ_truck_acc_data["TM_Y_CRD"].values.reshape(-1,1)

# create model and prediction
dbscan = DBSCAN(eps=0.00003,min_samples=3)
clusters = dbscan.fit_predict(x,y)

DJ_truck_acc_data["cluster"] = dbscan.labels_

unique, counts = np.unique(dbscan.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)

[[  -1 1294]
 [   0    6]
 [   1    4]
 [   2   12]
 [   3    6]
 [   4    3]
 [   5    3]
 [   6    7]
 [   7    7]
 [   8   12]
 [   9    4]
 [  10    5]
 [  11    3]
 [  12   87]
 [  13    3]
 [  14   25]
 [  15    6]
 [  16    9]
 [  17    3]
 [  18   11]
 [  19    3]
 [  20    6]
 [  21    8]
 [  22    4]
 [  23    6]
 [  24    3]
 [  25   22]
 [  26    8]
 [  27    7]
 [  28    4]
 [  29    7]
 [  30    6]
 [  31   14]
 [  32   15]
 [  33    5]
 [  34    3]
 [  35    4]
 [  36    7]
 [  37   18]
 [  38    3]
 [  39   14]
 [  40    4]
 [  41    4]
 [  42    5]
 [  43    4]
 [  44    9]
 [  45    4]
 [  46   18]
 [  47    3]
 [  48    3]
 [  49    4]
 [  50    3]
 [  51    4]
 [  52    3]
 [  53    4]
 [  54    4]
 [  55    7]
 [  56    4]
 [  57    5]
 [  58    3]
 [  59    4]
 [  60    3]
 [  61   22]
 [  62    5]
 [  63    3]
 [  64   12]
 [  65    6]
 [  66    5]
 [  67    7]
 [  68    8]
 [  69    4]
 [  70    4]
 [  71    6]
 [  72    3]
 [  73    4]
 [  74    7]
 [  75    9]

In [ ]:
###대전광역시_법정경계 Data
DJ_region = gpd.read_file("/home/yh_zoo/data/SBJ_2102_003/25.대전광역시_법정경계(시군구).geojson")

###기본 map설정
m = folium.Map(location=[36.3421127852882, 127.39291802200864],
               zoom_start=11)

#법정경계 표시
folium.Choropleth(geo_data=DJ_region["geometry"],
                  fill_color="black",
                  ).add_to(m)

#사고 시각화 circle
for i in range(len(DJ_truck_acc_data)):
    folium.Circle([DJ_truck_acc_data["TM_X_CRD"].iloc[i],
                   DJ_truck_acc_data["TM_Y_CRD"].iloc[i]],
                  radius=3,
                #  key_on='feature.properties.cluster', # 여기가 바뀌었습니다.
                #  fill_color='BuPu',
                  color=colors[(DJ_truck_acc_data["cluster"].iloc[i])],
                  fill_color=colors[(DJ_truck_acc_data["cluster"].iloc[i])]
                  ).add_to(m)
    
m

### grid 기반 (100x100)

In [ ]:
###대전광역시_Grid map
DJ_grid = gpd.read_file("/home/yh_zoo/data/SBJ_2102_003/30.대전광역시_차량등록현황_격자.geojson").drop("cars_cnt",axis=1)

###기본 map설정
m = folium.Map(location=[36.3421127852882, 127.39291802200864],
               zoom_start=11)

#법정경계 표시
folium.Choropleth(geo_data=DJ_grid["geometry"],
                  fill_color="#black",
                  ).add_to(m)

#사고 시각화 circle
for i in range(len(DJ_truck_acc_data)):
    folium.Circle([DJ_truck_acc_data["TM_X_CRD"].iloc[i],
                   DJ_truck_acc_data["TM_Y_CRD"].iloc[i]],
                  radius=3,
                #  key_on='feature.properties.cluster', # 여기가 바뀌었습니다.
                #  fill_color='BuPu',
                  color="red",
                  fill_color=colors[(DJ_truck_acc_data["cluster"].iloc[i])]
                  ).add_to(m)
    
m

- 사고에 대한 미래 예측 
    - 1일 기준 사고 예측
    - 사고 전에 알 수 있는 변수만 사용
    - Year, Month, Day, Week of day(요일), WEA_STA_CD(날씨), BJD_CD(법정동 코드), RD_TYP_CD(도로 종류 코드), RD_TYP_CD(도로 형태 코드)

In [18]:
DJ_truck_acc_data = pd.read_csv("/home/yh_zoo/data/prepared_acc_data.csv")
display(DJ_truck_acc_data.shape,DJ_truck_acc_data.head())

(3155, 16)

,Unnamed: 0,ACC_YMD,ACC_TME,DAY_CD,BJD_CD,ACC_TYP_CD,LAW_VIO_CD,TM_X_CRD,TM_Y_CRD,WEA_STA_CD,RD_CLS_CD,RD_TYP_CD,ACC_GRD_CD,DEATH_CNT,SEX_CD,AGE_CD
0,0,20170211,1300,7,3017012800,220,7.0,36.369776,127.379973,1,3.0,8.0,3,0,01,56
1,1,20160228,300,1,3014011700,310,6.0,36.309750,127.387941,1,3.0,2.0,3,0,01,23
2,2,20160205,300,6,3020012400,210,2.0,36.374644,127.379498,1,3.0,1.0,3,0,01,56
3,3,20160202,2200,3,3020010200,210,2.0,36.311836,127.317236,1,3.0,8.0,3,0,01,32
4,4,20160204,800,5,3020013900,240,6.0,36.391700,127.314986,1,3.0,8.0,3,0,01,33


In [19]:
###point feature 추가
DJ_truck_acc_data["Point"] = [Point(DJ_truck_acc_data["TM_Y_CRD"].iloc[i],DJ_truck_acc_data["TM_X_CRD"].iloc[i])for i in range(len(DJ_truck_acc_data))]

/home/yh_zoo/.local/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [20]:
###select colunms
data = DJ_truck_acc_data.drop(["Unnamed: 0","ACC_TYP_CD", "LAW_VIO_CD","ACC_GRD_CD","DEATH_CNT","SEX_CD","AGE_CD","TM_X_CRD","TM_Y_CRD","ACC_TME","BJD_CD","RD_TYP_CD","RD_CLS_CD"],axis=1)
data.shape,data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3155 entries, 0 to 3154
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ACC_YMD     3155 non-null   int64 
 1   DAY_CD      3155 non-null   int64 
 2   WEA_STA_CD  3155 non-null   int64 
 3   Point       3155 non-null   object
dtypes: int64(3), object(1)
memory usage: 98.7+ KB


((3155, 4), None)

In [21]:
###check null value
data.shape,data.isnull().sum()

((3155, 4),
 ACC_YMD       0
 DAY_CD        0
 WEA_STA_CD    0
 Point         0
 dtype: int64)

In [135]:
###사고 별 고유 격자 배정하기 (이중 for문 싫은데... 다른 방법이 없나... 병렬처리 안되나... QGIS써야되나..)
point_list = []

for id,i in enumerate(tqdm(data["Point"])):
    for idx,j  in enumerate(DJ_grid["geometry"]):
        if i.within(j):
            point_list.append(DJ_grid["gid"][idx])
    if len(point_list) != id+1:
        point_list.append("0")

###고유 격자 할당        
data["grid"] = point_list

###grid으로 하여 왹곽 지역이 grid에 포함 안됨... -> 4곳만 있으니 제거 
data = data.loc[data["grid"] != "0"].reset_index(drop=True)

100%|██████████| 3155/3155 [06:39<00:00,  7.90it/s]


In [136]:
###날짜 확인 : 날짜,고유 grid 기준 데이터 생성
min(data.ACC_YMD), max(data.ACC_YMD)

(20160101, 20191231)

In [137]:
from datetime import datetime, timedelta

###날짜 생성 함수 (시작날짜, 마지막날짜)
def date_range(start, end):
    start = datetime.strptime(start, "%Y-%m-%d")
    end = datetime.strptime(end, "%Y-%m-%d")
    dates = [(start + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((end-start).days+1)]
    return dates

dates = date_range("2016-01-01", "2019-12-31")

###날짜 필요한 형식으로 변환
def date_split(data):
    date = data.split("-")[0] + data.split("-")[1] + data.split("-")[2]
    date = int(date)
    return date

###dates
dates = [date_split(i) for i in dates]
dates = pd.DataFrame(dates,columns=["ymd"])

###grid
grid = DJ_grid[["gid"]]

###날짜 x  grid 모든 조합 base_data 만들기
import itertools

info = [dates.values, grid.values]

base_data = pd.DataFrame(itertools.product(*info), columns=["ymd","grid"])

base_data["ymd"] = [i[0] for i in base_data["ymd"]]
base_data["grid"] = [i[0] for i in base_data["grid"]]
base_data.head()

,ymd,grid
0,20160101,다마846996
1,20160101,다마847995
2,20160101,다마847996
3,20160101,다마847997
4,20160101,다마847998


In [138]:
#DJ_truck_acc_data 정보 저장
acc_info = {}

for i in data.groupby("ACC_YMD"):
    acc_info[i[0]] ={"DAY_CD":i[1].DAY_CD.values,"WEA_STA_CD":i[1].WEA_STA_CD.values,"GRID":i[1].grid.values}

#추가 컬럼 만들기
base_data["DAY_CD"] = 0
base_data["cnt"] = 0
base_data["WEA_STA_CD"] = 0

In [141]:
acc_info[20160101]

{'DAY_CD': array([6, 6], dtype=int64),
 'WEA_STA_CD': array([1, 1], dtype=int64),
 'GRID': array(['다바905129', '다바936147'], dtype=object)}

In [139]:
base_data_ = base_data.groupby("ymd")

base_data_list = []

for i in tqdm(base_data_):
    ymd = i[0]
    sub_data = i[1]
    try:
        ##DAY_CD
        sub_data["DAY_CD"] = acc_info[ymd]["DAY_CD"][0]
        ##WEA_STA_CD
        sub_data["WEA_STA_CD"] = acc_info[ymd]["WEA_STA_CD"][0]
        ##acc_cnt
        sub_gid = pd.DataFrame(pd.DataFrame(acc_info[ymd]["GRID"])[0].value_counts()).reset_index() 
        sub_gid.columns = ["gid","cnt"]
        ymd_gid = pd.merge(grid, sub_gid, how="left")
        ymd_gid = ymd_gid.fillna(0)
        sub_data["cnt"] = ymd_gid["cnt"].values
    except:
        pass
    base_data_list.append(sub_data)
    
prepared_data = pd.concat(base_data_list,axis=0)

###DAY_CD = 0 인 데이터 삭제 다른 데이터도 없기 때문
prepared_data = prepared_data.loc[prepared_data["DAY_CD"] != 0]

###year, month, day features 생성
prepared_data["year"] = [int(str(i)[:4]) for i in prepared_data["ymd"]]
prepared_data["month"] = [int(str(i)[4:6]) for i in prepared_data["ymd"]]
prepared_data["day"] = [int(str(i)[6:8]) for i in prepared_data["ymd"]]

pre_data = prepared_data[["year","month","day","DAY_CD","WEA_STA_CD","grid","cnt"]]

###시각화를 위해 grid 따로 빼놓기
pre_data.head()

pre_data.to_csv("data/prepared_train.csv",index=False)

100%|██████████| 1461/1461 [00:26<00:00, 54.61it/s]


## modeling
- target : cnt

- train, valid, test data 만들기 (시계열 순으로 잘라서 하기)

In [29]:
pre_data = pd.read_csv("data/prepared_train.csv")

In [30]:
one_day_grid = len(set(pre_data["grid"]))

In [42]:
X = pre_data.drop(["Unnamed: 0","cnt"], axis=1)
y = pre_data["cnt"].values

x_train_full, x_test, y_train_full, y_test = X[:1000*one_day_grid],X[1000*one_day_grid:], y[:1000*one_day_grid],y[1000*one_day_grid:]
x_train, x_valid, y_train, y_valid = x_train_full[:800*one_day_grid], x_train_full[800*one_day_grid:], y_train_full[:800*one_day_grid], y_train_full[800*one_day_grid:]

-  modeling

In [51]:
import catboost as cb

cat_features_index = [5]

clf = cb.CatBoostClassifier(eval_metric="AUC", depth=10, iterations= 100, learning_rate= 0.0001, task_type = 'GPU')
clf.fit(x_train,y_train, eval_set=(x_valid, y_valid), cat_features= cat_features_index)

AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.1734477	best: 0.1734477 (0)	total: 830ms	remaining: 1m 22s
1:	test: 0.1743003	best: 0.1743003 (1)	total: 1.36s	remaining: 1m 6s
2:	test: 0.1743003	best: 0.1743003 (1)	total: 1.94s	remaining: 1m 2s
3:	test: 0.1743003	best: 0.1743003 (1)	total: 2.48s	remaining: 59.4s
4:	test: 0.1721791	best: 0.1743003 (1)	total: 3.04s	remaining: 57.7s
5:	test: 0.1729740	best: 0.1743003 (1)	total: 3.62s	remaining: 56.7s
6:	test: 0.1724666	best: 0.1743003 (1)	total: 4.16s	remaining: 55.3s
7:	test: 0.1723782	best: 0.1743003 (1)	total: 4.74s	remaining: 54.5s
8:	test: 0.1723782	best: 0.1743003 (1)	total: 5.28s	remaining: 53.4s
9:	test: 0.1720956	best: 0.1743003 (1)	total: 5.84s	remaining: 52.6s
10:	test: 0.1726303	best: 0.1743003 (1)	total: 6.4s	remaining: 51.8s
11:	test: 0.1719142	best: 0.1743003 (1)	total: 6.97s	remaining: 51.2s
12:	test: 0.1719142	best: 0.1743003 (1)	total: 7.51s	remaining: 50.3s
13:	test: 0.1713104	best: 0.1743003 (1)	total: 8.08s	remaining: 49.6s
14:	test: 0.1717054	best: 0.17